In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv("/kaggle/input/student-performance-data-set/student-por.csv")
data.head()

In [ ]:
data["G_Total"]=(data["G1"]+data["G2"]+data["G3"])/3

In [ ]:
data.corr()

In [ ]:
data.describe()

In [ ]:
x=data.iloc[:,0:30]
y=data["G_Total"]

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [ ]:
from sklearn import preprocessing
le_mjob = preprocessing.LabelEncoder()
x_train["Mjob"]=le_mjob.fit_transform(x_train["Mjob"])
x_test["Mjob"]=le_mjob.transform(x_test["Mjob"])

In [ ]:
le_fjob = preprocessing.LabelEncoder()
x_train["Fjob"]=le_fjob.fit_transform(x_train["Fjob"])
x_test["Fjob"]=le_fjob.transform(x_test["Fjob"])

In [ ]:
x_train[["Fjob","Mjob"]].corr()

Mother job is not correlated with Father job. It is maybe a information. 

In [ ]:
le_famsize = preprocessing.LabelEncoder()
x_train["famsize"]=le_famsize.fit_transform(x_train["famsize"])
x_test["famsize"]=le_famsize.transform(x_test["famsize"])

Okay I don't want to use gender or school columns. 

In [ ]:
X_train=x_train._get_numeric_data()
X_test=x_test._get_numeric_data()

In [ ]:
import xgboost as xgb
xgbreg=xgb.XGBRegressor()

from sklearn.model_selection import cross_val_score
cross_val_score(xgbreg,X_train,y_train,cv=5,scoring="neg_mean_absolute_error")

In [ ]:
y_train.describe()

XGBRegressor model is works okay. It looks acceptable.

In [ ]:
xgbreg=xgb.XGBRegressor()
xgbreg.fit(X_train,y_train)
ypred=xgbreg.predict(X_test)

import sklearn.metrics as metrik
metrik.mean_absolute_error(y_true=y_test,y_pred=ypred)

Okay we simplified the model. Less COlumns maybe improve our model.

In [ ]:
from sklearn.feature_selection import SelectKBest,mutual_info_regression
selek8=SelectKBest(mutual_info_regression, k=8)

X8_train=selek8.fit_transform(X_train,y_train)
X8_test=selek8.transform(X_test)

xgbreg=xgb.XGBRegressor()
xgbreg.fit(X8_train,y_train)
ypred=xgbreg.predict(X8_test)

import sklearn.metrics as metrik
metrik.mean_absolute_error(y_true=y_test,y_pred=ypred)

In [ ]:
selek8.get_support()

In [ ]:
from sklearn.feature_selection import SelectKBest,mutual_info_regression
selek1=SelectKBest(mutual_info_regression, k=1)

X1_train=selek1.fit_transform(X_train,y_train)
X1_test=selek1.transform(X_test)

xgbreg=xgb.XGBRegressor()
xgbreg.fit(X1_train,y_train)
ypred=xgbreg.predict(X1_test)

import sklearn.metrics as metrik
metrik.mean_absolute_error(y_true=y_test,y_pred=ypred)

In [ ]:
i=0
for column in  list(selek1.get_support()):
    if column:
        print(i)
        break
    i=i+1    

In [ ]:
x_train.iloc[:,8:9]

Mjob is highly correlated with Grades. İt is double interesting.

Okay drop the MJOB column

In [ ]:
X_new_train=X_train.drop("Mjob",axis=1)
X_new_test=X_test.drop("Mjob",axis=1)

xgbreg=xgb.XGBRegressor()

from sklearn.model_selection import cross_val_score
cross_val_score(xgbreg,X_new_train,y_train,cv=5,scoring="neg_mean_absolute_error")

In [ ]:
xgbreg=xgb.XGBRegressor()
xgbreg.fit(X_new_train,y_train)
ypred=xgbreg.predict(X_new_test)

import sklearn.metrics as metrik
metrik.mean_absolute_error(y_true=y_test,y_pred=ypred)

In [ ]:
x_train.iloc[:,7:8]

In [ ]:
X_new_train=X_train.drop("Fedu",axis=1)
X_new_test=X_test.drop("Fedu",axis=1)

xgbreg=xgb.XGBRegressor()

from sklearn.model_selection import cross_val_score
print(cross_val_score(xgbreg,X_new_train,y_train,cv=5,scoring="neg_mean_absolute_error"))

xgbreg=xgb.XGBRegressor()
xgbreg.fit(X_new_train,y_train)
ypred=xgbreg.predict(X_new_test)

import sklearn.metrics as metrik
print(metrik.mean_absolute_error(y_true=y_test,y_pred=ypred))

In [ ]:
X_new_train=X_train.drop(["Fedu","Mjob"],axis=1)
X_new_test=X_test.drop(["Fedu","Mjob"],axis=1)

xgbreg=xgb.XGBRegressor()

from sklearn.model_selection import cross_val_score
print(cross_val_score(xgbreg,X_new_train,y_train,cv=5,scoring="neg_mean_absolute_error"))

xgbreg=xgb.XGBRegressor()
xgbreg.fit(X_new_train,y_train)
ypred=xgbreg.predict(X_new_test)

import sklearn.metrics as metrik
print(metrik.mean_absolute_error(y_true=y_test,y_pred=ypred))

OKay examination needed and also hyperparamter tuning is needed. 

In [ ]:
mjob_train=X_train[["Mjob"]]
mjob_test=X_test[["Mjob"]]

xgbreg=xgb.XGBRegressor()

from sklearn.model_selection import cross_val_score
print(cross_val_score(xgbreg,mjob_train,y_train,cv=5,scoring="neg_mean_absolute_error"))

In [ ]:
xgbreg=xgb.XGBRegressor()
xgbreg.fit(mjob_train,y_train)
ypred=xgbreg.predict(mjob_test)

import sklearn.metrics as metrik
print(metrik.mean_absolute_error(y_true=y_test,y_pred=ypred))

In [ ]:
set(ypred)

OKay PCA with SVM Maybe work better.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=8, svd_solver='full')
X_train_pca=pca.fit_transform(X_train)
X_test_pca=pca.transform(X_test)
from sklearn.svm import SVR
print(cross_val_score(SVR(),X_train_pca,y_train,cv=5,scoring="neg_mean_absolute_error"))

In [ ]:
svr=SVR()
svr.fit(mjob_train,y_train)
ypred=svr.predict(mjob_test)

import sklearn.metrics as metrik
print(metrik.mean_absolute_error(y_true=y_test,y_pred=ypred))

In [ ]:
pd.Series(ypred).describe()

Similar value. XGBoost and SVR can't produce higher and lower values as expectedly.

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
C_range = np.logspace(-2, 10, 4)
gamma_range = np.logspace(-9, 3, 4)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVR(kernel="rbf"), param_grid=param_grid,n_jobs=5,scoring="neg_root_mean_squared_error")
grid.fit(X_train_pca, y_train)

print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

In [ ]:
svr_tuned=SVR(C=1000000.0,gamma=1e-05,kernel="rbf")
svr_tuned.fit(mjob_train,y_train)
ypred=svr_tuned.predict(mjob_test)

import sklearn.metrics as metrik
print(metrik.mean_absolute_error(y_true=y_test,y_pred=ypred))

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
C_range = np.logspace(-2, 10, 6)
gamma_range = np.logspace(-9, 3, 6)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVR(kernel="rbf"), param_grid=param_grid,cv=3,n_jobs=5)
grid.fit(X_train_pca, y_train)

print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

In [ ]:
svr_tuned=SVR(C=158489.3192461111,gamma=2.5118864315095823e-07,kernel="rbf")
svr_tuned.fit(mjob_train,y_train)
ypred=svr_tuned.predict(mjob_test)

import sklearn.metrics as metrik
print(metrik.mean_absolute_error(y_true=y_test,y_pred=ypred))

In [ ]:
pd.Series(ypred).describe()

OKay nothing change.

In [ ]:
pd.Series(y_test).describe()

Can't goes to 1 or 18. Okay plain Decision Tree maybe work fine.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtree=DecisionTreeRegressor()
print(cross_val_score(dtree,X_train,y_train,cv=5,scoring="neg_mean_absolute_error"))

In [ ]:
dtree=DecisionTreeRegressor()
dtree.fit(X_train,y_train)
ypred=dtree.predict(X_test)

import sklearn.metrics as metrik
print(metrik.mean_absolute_error(y_true=y_test,y_pred=ypred))

In [ ]:
pd.Series(ypred).describe()

OKay Decision Tree works better. Have 2s and up to 17. Didn't stuck in 10-12. **Best working model is Decision Tree.**

In [ ]:
criterion = ["mse", "friedman_mse", "mae"]
max_depth = [2,3,4,6,8,10,12,13]
max_features=["auto","sqrt","log2"]
param_gird = dict(criterion=criterion,max_features=max_features,
                      max_depth=max_depth)

grid = GridSearchCV(DecisionTreeRegressor(), param_grid=param_gird,cv=3,n_jobs=5,scoring="neg_root_mean_squared_error")
grid.fit(X_train, y_train)

print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

In [ ]:
dtree=DecisionTreeRegressor(criterion= "mae",max_depth= 2, max_features= 'auto')
dtree.fit(X_train,y_train)
ypred=dtree.predict(X_test)

import sklearn.metrics as metrik
print(metrik.mean_absolute_error(y_true=y_test,y_pred=ypred))

In [ ]:
import pickle

filename = 'dtreemodel.pkl'
pickle.dump(dtree, open(filename, 'wb'))